# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/gemma-3n-E4B-it',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/OmniGen2/OmniGen2',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/facebook/seamless-interaction',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/nvidia/OpenScience',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tran

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [30]:
print(get_all_details("https://www.walkerphotography.org"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.walkerphotography.org/about'}, {'type': 'contact page', 'url': 'https://www.walkerphotography.org/contact'}, {'type': 'galleries page', 'url': 'https://www.walkerphotography.org/galleries'}]}
Landing page:
Webpage Title:
Family of photographers lovers | Walkerphotography
Webpage Contents:
Home
About
Blog
Contact
Galleries
Wildlife
Landscapes
Architecture
Street / Life
Home
About
Blog
Contact
Galleries
Wildlife
Landscapes
Architecture
Street / Life
With each photograph, we capture the magic of light, the tranquility of nature and the pulse of life.
Find out more
Galleries
Wildlife
Landscapes
Architecture
Street
©
2025 Walkerphotography. All rights reserved.



about page
Webpage Title:
About the Walker family | Walkerphotography
Webpage Contents:
Home
About
Blog
Contact
Galleries
Wildlife
Landscapes
Architecture
Street / Life
Home
About
Blog
Contact
Galleries
Wildlife
Landscapes
Architecture
Street / Life
Get to know us


In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n6 days ago\n•\n144k\n•\n1.27k\ntencent/Hunyuan-A13B-Instruct\nUpdated\n2 days ago\n•\n9.23k\n•\n697\ngoogle/gemma-3n-E4B-it\nUpdated\nabout 20 hours ago\n•\n175k\n•\n420\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n1 day ago\n•\n4.12k\n•\n175\nOmniGen2/OmniGen2\nUpdated\n10 days ago\n•\n34.6k\n•\n335\nBrowse 1M+ models\nSpaces\nRunning\n9.19k\n9.19k\

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'social media page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Welcome to Hugging Face! 🤗

**Where the Robots Hold the Future's Hands!**  

At Hugging Face, we’re not just building cutting-edge AI models; we’re shaping a future where machines and humans coexist (preferably without any robot uprisings). Our open-source community brings together the most brilliant minds in machine learning to create, collaborate, and occasionally confuse the robots with jokes.

## What Do We Offer? 🛠️

- **1M+ Models** 📊: Browse our library that's just about as big as your in-laws' collection of holiday photos! From image generation to text analysis, we’ve got a model for every task (even if that task is just generating awkward small talk).
  
- **Datasets Galore!** 🗃️: Dive into over **250k datasets** because why stop at just one when you can have them all? It's like a buffet, but for data—bring your biggest storage drive!

- **Spaces** 🏢: Create and share applications effortlessly in our user-friendly "Spaces." Think of it as the 'cool kids' playroom for AI enthusiasts where the only rule is to not talk about Fight Club.

## Our Culture 🌈

At Hugging Face, we believe in the power of collaboration over competition! Picture a cozy place where everyone’s on a first-name basis with the code. We value creativity, initiative, and the occasional virtual high-five! Join an environment where machine learning isn't work—it's more like a growing family of nerds who can't stop hugging data (and each other).

- **Team Spirit**: Collaborate with **50,000+ organizations** (yes, we’re a big deal)! From tech giants like Google and Amazon to charming little startups like “Your Mom's Kitchen,” our community is diverse, and so is our choice of snacks.

## Careers at Hugging Face 🚀

Ready to join us? We're always on the lookout for talented individuals who love AI nearly as much as they love coffee (because let’s face it, the latter keeps you awake to figure out why your model isn't working)! Check out our latest job openings and see if you have what it takes to join our cool club!

### Current Openings
- Model Wrangler (just kidding, but we do have some fun jobs!)
- Dataset Tamer (you'll never have a dull moment!)
- Hugging Specialist (this one’s for real—bring your best hugs!)

## Our Customers (aka Awesome People) 🤝

We've partnered with **companies that are kinda a big deal**—Meta, Amazon, and Google, to name a few. But our true customers are the unpredictable geniuses in the community who play with our tools, tweak our models, and make us rethink our entire existence (no pressure!).

## Join Us! 🙌

Want to nurture the future of AI? Visit our website to explore models, datasets, and apply for a position. We promise you’ll leave with a smile (and maybe a model or two).

**Hugging Face – Because AI is better with hugs.** 🤗💙

[Explore More](#)
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face! 🤗

**The AI Community Building the Future**  
Where dreams become AI reality, and where everyone gets a hug! (Disclaimer: Hugs may be virtual in nature.)

---

## Who Are We?

Picture a group of machine learning enthusiasts, some with caffeine-induced superpowers, all collaborating to create a better tomorrow through AI goodness! From models to datasets, we’ve got it all—1 Million+ models and counting! We might even hold a world record for the most caffeinated brains in one online space. ☕

---

## Our Customers & Partners 🤝

We're on the guest list for more than **50,000 organizations**! That’s right, you’re in good company! From **Amazon** to **Google** to **Grammarly**: we like to call ourselves the “Hugging Network of Champions.” So grab your cape, and let us save the day—one AI model at a time! 🦸‍♂️

---

## Why Choose Us?

- **Models Galore**: Stuck in a model rut? We have 1M+ models to explore—just remember to bring your adventure hat! 🎩
  
- **Datasets to Die For**: Curious about the latest datasets? We've got over **250k** waiting to be uncovered. They shine like diamonds! 💎

- **Community Love**: We put the “unity” in community! Join us in our collaborative chaos where every update feels like a group hug.

---

## Our Amazing Team Culture 🎉

At Hugging Face, the coffee flows as freely as the ideas! We embrace flexible schedules, quirky memes, and the collective belief that a good dad joke can solve nearly any problem. "Why did the neural network break up with its girlfriend? She said it had too many layers!" 🥁

### Career Opportunities

Are you ready to transform your love for machine learning into a fun job? We're looking for:
- **Model Magicians**: Craft enchanting models that leave everyone spellbound.
- **Dataset Adventurers**: Navigate through datasets like Indiana Jones.
- **AI Ambassadors**: Spread the word about our AI superpowers—one hug at a time!

---

## How to Join Us! 🚀

Could you be the next AI superhero in our community? Joining is simple! Sign up on our website, and get ready to collaborate, innovate, and give virtual high-fives!

| **Sign Up Now** | **Get a Hug** |
|------------------|---------------|
| [Hugging Face Sign Up](https://huggingface.co) | 🤗 |

---

Remember, whether you're a customer, investor, or future recruit—when you're with Hugging Face, you're family. The world of AI awaits, and we can’t wait to see what we’ll build together!

---

*Note: Side effects of joining Hugging Face may include excessive hugging, sudden bursts of creativity, and frequent “pants-optional” meetings (just kidding?).* 🤔



In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🎉 Welcome to Hugging Face! 🎉

**The AI Community Building the Future!**  
(Disclaimer: Hugging Face does not provide actual hugs but does encourage the sharing of knowledge and collaboration—keep your arms to yourself!)

---

## Who We Are 🤖

At **Hugging Face**, we're not just building models, datasets, and apps—we're creating a community (don’t worry, no awkward small talk required). With over **1M+ models** and **250k+ datasets**, we’re the cool kids on the AI block!

We unite developers, data scientists, and AI enthusiasts under the banner of innovation, collaboration, and a sprinkling of humor! Our ultimate goal? To help dial down the "AI is taking over the world" panic and instead, make everyone say, “Wow, AI is like my loyal pet that brings me coffee!” ☕

---

## Our Customers 👩‍💻👨‍💼

Join **50,000+ organizations** already tapping into our eccentric universe, including:

- **Meta** – Because why have an average corporate life when you can work on AI?
- **Amazon** – Crafting the future of shopping, one ML model at a time!
- **Google** – For when search engines want to sound smart at parties.
- **Microsoft** – Fixing bugs while wearing their iconic “I'm an AI wizard” hats. 🍵🧙

---

## Join the Fun! Careers at Hugging Face 🌟

Looking to turn that empty coffee cup into a full-time gig? Check out **Hugging Face** for job openings! We’re embracing talents with open arms (and slightly less creepy than an actual hug). Whether you're a model trainer, a data wrangler, or just an enthusiastic AI lover, we might have the spot for you!

### Current Openings:
- **AI Wizards**: Get ready to conjure magic from data.
- **Community Gurus**: Make sure our community feels loved (with virtual hugs).
- **Tech Ninjas**: Your stealth skills with code can help us dance through challenges!

---

## Our Culture 🍕 & 🤝

At **Hugging Face**, we believe in open-source everything! Imagine a world where colleagues share pizza and insightful AI models. Our workflow is open, our minds are broader, and our coffee breaks are legendary. Trust us; you'll never hear "There's no 'I' in team" here because we're too busy collaborating to notice! 

---

## Let’s Get Started! 🚀

Ready to join a community that’s building the future with algorithms and great coffee? Whether you’re a customer looking for AI solutions or a candidate with a thirst for innovation, **Hugging Face** is the place where dreams (and datasets) come true! 🥳 

#### Explore. Collaborate. Hug (virtually)!

For more information, visit us at [Hugging Face](https://huggingface.co) (valid URL disclaimer!)

---

*Disclaimer: No actual hugging was involved in the creation of this brochure; all models were generated in the spirit of affection.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>